In [1]:
import os
import cv2
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [2]:
import os
from zipfile import ZipFile


!pip install kaggle


os.environ['KAGGLE_USERNAME'] = "majisouvik1099"
os.environ['KAGGLE_KEY'] = "9464225218d56f8bfea9dd9cc437489f"


!kaggle datasets download -d jessicali9530/lfw-dataset


with ZipFile('lfw-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('lfw-dataset')


os.remove('lfw-dataset.zip')


 89% 100M/112M [00:00<00:00, 159MB/s] 
100% 112M/112M [00:00<00:00, 148MB/s]


In [3]:

def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    features = features.squeeze(0)
    return features


# def compute_hog(img):
#     resized_img = resize(img, (128*4, 64*4))
#     fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
#                     cells_per_block=(2, 2), visualize=True, multichannel=True)
#     return fd


def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

# def lbp_calculated_pixel(img, x, y):
#     center = img[x][y]
#     val_ar = []
#     val_ar.append(get_pixel(img, center, x-1, y+1))
#     val_ar.append(get_pixel(img, center, x, y+1))
#     val_ar.append(get_pixel(img, center, x+1, y+1))
#     val_ar.append(get_pixel(img, center, x+1, y))
#     val_ar.append(get_pixel(img, center, x+1, y-1))
#     val_ar.append(get_pixel(img, center, x, y-1))
#     val_ar.append(get_pixel(img, center, x-1, y-1))
#     val_ar.append(get_pixel(img, center, x-1, y))

#     power_val = [1, 2, 4, 8, 16, 32, 64, 128]
#     val = 0
#     for i in range(len(val_ar)):
#         val += val_ar[i] * power_val[i]
#     return val

# def calcLBP(img):
#     height, width, channel = img.shape
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img_lbp = np.zeros((height, width,3), np.uint8)
#     for i in range(0, height):
#         for j in range(0, width):
#              img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
#     hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
#     return hist_lbp.flatten()


In [4]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 69.1MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [5]:
lfw_folder = '/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
X, y = [], []
for folder_name in os.listdir(lfw_folder):
    folder_path = os.path.join(lfw_folder, folder_name)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        if num_images > 70:
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                image = imread(image_path)
                # hog_feature = compute_hog(image)
                # lbp_feature = calcLBP(image)
                cnn_feature = extract_features(image_path, resnet).numpy()
                # hog_feature = hog_feature.reshape(-1)
                # lbp_feature = lbp_feature.reshape(-1)
                cnn_feature = cnn_feature.flatten()
                # combined_feature = np.concatenate((hog_feature, lbp_feature, cnn_feature))

                X.append(cnn_feature)
                y.append(folder_name)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [7]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [30]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["linear"],
            "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]},
        {"kernel":
            ["poly"],
            "degree": [2, 3, 4],
            "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]},
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


grid search best score: 79.22%
grid search best parameters: {'C': 10, 'degree': 2, 'kernel': 'poly'}


In [31]:
model = grid.best_estimator_
y_pred_svm = model.predict(X_test)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.69      0.82      0.75        11
     Colin_Powell       0.85      0.81      0.83        42
  Donald_Rumsfeld       0.48      0.60      0.53        20
    George_W_Bush       0.87      0.87      0.87       126
Gerhard_Schroeder       0.74      0.61      0.67        23
      Hugo_Chavez       0.47      0.64      0.54        11
       Tony_Blair       0.75      0.60      0.67        25

         accuracy                           0.78       258
        macro avg       0.69      0.71      0.69       258
     weighted avg       0.79      0.78      0.78       258



In [34]:
y_pred_svm = model.predict(X_train)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Training Accuracy:", accuracy_svm)

SVM Classifier
Training Accuracy: 1.0


In [35]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

model.fit(X_train_pca, y_train_encoded)
y_pred = model.predict(X_test_pca)
accuracy = accuracy_score(y_test_encoded, y_pred)
print("SVM Classifier")
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))

SVM Classifier
Accuracy: 0.6627906976744186
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.50      0.36      0.42        11
     Colin_Powell       0.67      0.67      0.67        42
  Donald_Rumsfeld       0.69      0.55      0.61        20
    George_W_Bush       0.70      0.83      0.76       126
Gerhard_Schroeder       0.53      0.35      0.42        23
      Hugo_Chavez       0.38      0.27      0.32        11
       Tony_Blair       0.62      0.52      0.57        25

         accuracy                           0.66       258
        macro avg       0.58      0.51      0.54       258
     weighted avg       0.65      0.66      0.65       258



In [39]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=None)
X_train_lda = lda.fit_transform(X_train, y_train_encoded)
X_test_lda = lda.transform(X_test)

model.fit(X_train_lda, y_train_encoded)
y_pred_svm = model.predict(X_test_lda)
accuracy_svm = accuracy_score(y_test_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Accuracy: 0.6046511627906976
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.50      0.55      0.52        11
     Colin_Powell       0.56      0.55      0.55        42
  Donald_Rumsfeld       0.52      0.65      0.58        20
    George_W_Bush       0.74      0.65      0.69       126
Gerhard_Schroeder       0.38      0.43      0.41        23
      Hugo_Chavez       0.50      0.55      0.52        11
       Tony_Blair       0.52      0.64      0.57        25

         accuracy                           0.60       258
        macro avg       0.53      0.57      0.55       258
     weighted avg       0.62      0.60      0.61       258

